# NextPrice

## 종목코드

In [21]:
import firm_code
firm_names = ['HLB']
firm_codes = firm_code.firm_codes(firm_names)
firm_codes = firm_code.filtering_firm_codes(firm_codes)
print(firm_codes)

['028300']


## FDR

In [22]:
import FinanceDataReader as fdr

In [23]:
df = fdr.DataReader(symbol=firm_codes[0])
df.tail(3)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2023-01-31,30650,31100,30150,30400,520871,-0.014587
2023-02-01,31000,33050,30900,31800,1411118,0.046053
2023-02-02,32250,32550,31600,31900,367329,0.003145


## Preprocessing

In [24]:
import nextprice_preprocessing

In [25]:
df = nextprice_preprocessing.moving_average(df, change=True)
df = nextprice_preprocessing.changing_price(df)
df = nextprice_preprocessing.create_y_label(df)
df = nextprice_preprocessing.slice_year(df,'2015')
X, y = split_x_y(df)

## Split

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Scaler

In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## PCA

In [28]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

## Model

In [29]:
from keras.optimizers import Adagrad
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense

In [30]:
model = Sequential()
model.add(Dense(12, input_dim=2, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(12, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adagrad(learning_rate=0.2)

In [31]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                36        
                                                                 
 dropout (Dropout)           (None, 12)                0         
                                                                 
 dense_1 (Dense)             (None, 12)                156       
                                                                 
 dropout_1 (Dropout)         (None, 12)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(X_train, y_train, epochs=40, batch_size=20)

Epoch 1/40
75/75 [==============================] - 1s 2ms/step - loss: 0.7001 - accuracy: 0.5369
Epoch 2/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6843 - accuracy: 0.5503
Epoch 3/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6864 - accuracy: 0.5576
Epoch 4/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6844 - accuracy: 0.5603
Epoch 5/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6815 - accuracy: 0.5670
Epoch 6/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6846 - accuracy: 0.5529
Epoch 7/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6851 - accuracy: 0.5536
Epoch 8/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6844 - accuracy: 0.5617
Epoch 9/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6837 - accuracy: 0.5583
Epoch 10/40
75/75 [==============================] - 0s 2ms/step - loss: 0.6828 - accuracy: 0.5610
Epoch 11/40
75/75 [

In [33]:
_, accuracy = model.evaluate(X_test, y_test)
print(accuracy)

16/16 [==============================] - 0s 2ms/step - loss: 0.6950 - accuracy: 0.5301
0.5301204919815063


In [34]:
y_pred = model.predict(X_test)

16/16 [==============================] - 0s 1ms/step


In [35]:
from sklearn import metrics
import numpy as np

In [36]:
confusion_matrix = metrics.confusion_matrix(y_test, np.rint(y_pred))
confusion_matrix

array([[218,  34],
       [200,  46]], dtype=int64)

## Save

In [37]:
import joblib
joblib.dump(scaler, './database/nextprice_scaler.pkl')

['./database/nextprice_scaler.pkl']

In [38]:
import pickle
pickle.dump(pca, open('./database/nextprice_pca.pkl','wb'))

In [39]:
from tensorflow import keras
model.save('./database/nextprice_model')

INFO:tensorflow:Assets written to: ./database/nextprice_model\assets


## Load

In [40]:
import joblib
scaler = joblib.load('./database/nextprice_scaler.pkl')

In [41]:
import pickle
pca = pickle.load(open('./database/nextprice_pca.pkl','rb'))

In [42]:
from tensorflow import keras
model = keras.models.load_model('./database/nextprice_model')

## Prediction

In [44]:
print(firm_codes[0])

028300


In [45]:
df = fdr.DataReader(symbol=firm_codes[0])
df = nextprice_preprocessing.moving_average(df, change=True)
df = nextprice_preprocessing.changing_price(df)
df = df[['MA5', 'MA20', 'MA60', 'MA120', 'Volume', 'Change', 'Open', 'High', 'Low']]
df = df[-1:]
df

,MA5,MA20,MA60,MA120,Volume,Change,Open,High,Low
Date,,,,,,,,,
2023-02-02,0.017925,0.025314,-0.001206,-0.14274,368551,0.0,0.014151,0.023585,-0.006289


In [46]:
df = scaler.transform(df)
df = pca.transform(df)

In [47]:
prediction = model.predict(df)
print(prediction)

1/1 [==============================] - 0s 73ms/step
[[0.40957013]]


## 최종코드